In [16]:
import pandas as pd
import numpy as np
import pickle
import os
import itertools

import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

In [6]:
data = pd.read_csv("../../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [14]:
texts = data.loc[0, "sentences"]
texts

['sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı',
 'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi',
 'polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı',
 'rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi',
 'polisin çalışması devam ediyor']

In [7]:
texts = data.loc[0, "sentences"]
texts

['sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı',
 'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi',
 'polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı',
 'rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi',
 'polisin çalışması devam ediyor']

In [29]:
texts_all = " [SEP] ".join(texts)
texts_all

'sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı [SEP] ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi [SEP] polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı [SEP] rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi [SEP] polisin çalışması devam ediyor'

In [23]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

In [24]:
tokenizer.cls_token_id, tokenizer.sep_token_id

(2, 3)

In [17]:
texts[1]

'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi'

In [37]:
tokenizer("[PAD]", truncation=True, padding='max_length', max_length=21)

{'input_ids': [2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [41]:
a = tokenizer(texts, truncation=True, padding=True)
b = [list(itertools.chain.from_iterable(a))[:-1]]
a, b

({'input_ids': [[2, 8449, 24635, 16217, 2446, 12088, 58520, 2676, 40082, 3574, 2095, 2058, 6512, 78175, 22704, 3149, 15279, 2189, 16217, 2932, 5735, 2065, 3149, 12874, 2789, 4190, 3730, 12305, 2386, 54080, 2956, 31083, 12088, 3], [2, 2683, 27631, 116288, 1019, 115973, 2280, 15279, 1011, 84087, 2023, 8546, 7641, 8059, 4190, 2426, 18517, 41569, 1971, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 11821, 5267, 29577, 53677, 18387, 2120, 4190, 21783, 2282, 52565, 4763, 12088, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 64557, 4190, 16853, 11649, 119506, 2537, 15279, 1011, 84087, 5326, 4673, 3985, 4341, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 11821, 52565, 2094, 2524, 2992, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [32]:
encodings = tokenizer(texts, truncation=True, padding=True)

In [67]:
len(encodings["input_ids"][3])

34

In [17]:
checkpoint = "dbmdz/bert-base-turkish-128k-uncased"
model = AutoModel.from_pretrained(checkpoint, config=AutoConfig.from_pretrained(checkpoint, output_attentions=True, output_hidden_states=True))

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [74]:
model.embeddings.word_embeddings.embedding_dim

768

In [58]:
output = model.embeddings(input_ids=torch.tensor(encodings['input_ids']))

In [60]:
output.shape

torch.Size([5, 34, 768])

In [79]:
output[:, [2, 3, 2, 4, 23], :]

tensor([[[-0.0032, -0.7094, -0.1654,  ..., -0.0210,  0.1837, -0.7734],
         [ 0.4957, -0.1394, -0.4093,  ...,  0.0177,  0.5172,  0.0981],
         [-0.0032, -0.7094, -0.1654,  ..., -0.0210,  0.1837, -0.7734],
         [-0.0297,  0.4567,  0.2448,  ..., -1.0937,  0.3745,  0.3019],
         [-0.0255,  0.9001, -0.3038,  ...,  0.6231, -0.1434,  0.3775]],

        [[-0.8814, -0.3384,  0.8060,  ..., -0.8117,  0.3281,  0.2105],
         [-0.2519,  0.4346,  0.3902,  ...,  0.4092, -0.4553,  0.4487],
         [-0.8814, -0.3384,  0.8060,  ..., -0.8117,  0.3281,  0.2105],
         [ 0.6175,  0.9882, -0.5111,  ..., -0.0976,  0.7252,  1.3528],
         [ 1.0049, -0.5554,  0.4313,  ...,  0.1391, -0.6563, -0.1494]],

        [[ 0.1365, -0.7286,  0.8736,  ...,  0.9784,  0.6735,  0.1406],
         [-0.7175, -0.1718,  0.7099,  ..., -0.2159, -0.0265,  0.1997],
         [ 0.1365, -0.7286,  0.8736,  ...,  0.9784,  0.6735,  0.1406],
         [ 0.0787, -0.2962, -0.7391,  ...,  0.0401, -0.2961, -0.6673],
  

In [50]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'attentions'])

In [53]:
output["pooler_output"].shape

torch.Size([5, 768])

In [56]:
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(128000, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)